In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from PIL import Image
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
#empty dataframe 
# data = pd.DataFrame(columns=['image_path', 'label'])
data_list = []

labels = {
    r"D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy": "Cloudy",
    r"D:\PYTHON\Edunet2.0\day-9\archive\data\green_area": "Green",
    r"D:\PYTHON\Edunet2.0\day-9\archive\data\water": "Water",
    r"D:\PYTHON\Edunet2.0\day-9\archive\data\desert": "Desert"
}

#validate the paths
for folder in labels:
    if not os.path.exists(folder):
        print(f"Directory {folder} does not exist.")
        continue
    for filename in os.listdir(folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder, filename)
            label = labels[folder]
            data_list.append({'image_path': image_path, 'label': label})

data = pd.DataFrame(data_list)
print(data)

                                             image_path   label
0     D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...  Cloudy
1     D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...  Cloudy
2     D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...  Cloudy
3     D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...  Cloudy
4     D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...  Cloudy
...                                                 ...     ...
5626  D:\PYTHON\Edunet2.0\day-9\archive\data\desert\...  Desert
5627  D:\PYTHON\Edunet2.0\day-9\archive\data\desert\...  Desert
5628  D:\PYTHON\Edunet2.0\day-9\archive\data\desert\...  Desert
5629  D:\PYTHON\Edunet2.0\day-9\archive\data\desert\...  Desert
5630  D:\PYTHON\Edunet2.0\day-9\archive\data\desert\...  Desert

[5631 rows x 2 columns]


In [6]:
data.to_csv("image_data.csv", index=False)

In [7]:
df = pd.read_csv('image_data.csv')
df

,image_path,label
0,D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...,Cloudy
1,D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...,Cloudy
2,D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...,Cloudy
3,D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...,Cloudy
4,D:\PYTHON\Edunet2.0\day-9\archive\data\cloudy\...,Cloudy
...,...,...
5626,D:\PYTHON\Edunet2.0\day-9\archive\data\desert\...,Desert
5627,D:\PYTHON\Edunet2.0\day-9\archive\data\desert\...,Desert
5628,D:\PYTHON\Edunet2.0\day-9\archive\data\desert\...,Desert
5629,D:\PYTHON\Edunet2.0\day-9\archive\data\desert\...,Desert


In [8]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])


In [9]:
train_Datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_Datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 4504 validated image filenames belonging to 4 classes.


In [11]:
cnnmodel  = Sequential()
cnnmodel.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Conv2D(64, (3, 3), activation='relu'))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Conv2D(128, (3, 3), activation='relu'))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Flatten())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dropout(0.5))
cnnmodel.add(Dense(4, activation='softmax'))
cnnmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnnmodel.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        1

In [14]:


history = cnnmodel.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
141/141 [==============================] - 130s 918ms/step - loss: 0.3296 - accuracy: 0.8577 - val_loss: 0.2672 - val_accuracy: 0.8900
Epoch 2/10
141/141 [==============================] - 130s 918ms/step - loss: 0.3296 - accuracy: 0.8577 - val_loss: 0.2672 - val_accuracy: 0.8900
Epoch 2/10
141/141 [==============================] - 120s 853ms/step - loss: 0.3378 - accuracy: 0.8521 - val_loss: 0.3000 - val_accuracy: 0.8873
Epoch 3/10
Epoch 3/10
141/141 [==============================] - 98s 694ms/step - loss: 0.3132 - accuracy: 0.8661 - val_loss: 0.2490 - val_accuracy: 0.9033
Epoch 4/10
141/141 [==============================] - 98s 694ms/step - loss: 0.3132 - accuracy: 0.8661 - val_loss: 0.2490 - val_accuracy: 0.9033
Epoch 4/10
141/141 [==============================] - 125s 887ms/step - loss: 0.2869 - accuracy: 0.8801 - val_loss: 0.3083 - val_accuracy: 0.8660
Epoch 5/10
Epoch 5/10
141/141 [==============================] - 103s 719ms/step - loss: 0.2903 - accuracy: 0.8777 